In [35]:
import math
import os
import requests
import cv2
import numpy as np
import warnings
import tqdm
warnings.filterwarnings("ignore")

In [36]:
# some utils
def build_url(x, y, z):
    return "http://0pn.cn/maps/vt?lyrs=s&x={x}&y={y}&z={z}".format(x=x, y=y, z=z)

def lonlat2xyz(lon, lat, zoom):
    n = math.pow(2, zoom)
    x = ((lon + 180) / 360) * n
    y = (1 - (math.log(math.tan(math.radians(lat)) + (1 / math.cos(math.radians(lat)))) / math.pi)) / 2 * n
    return int(x), int(y)

def xyz2lonlat(x, y, zoom):
    n = math.pow(2, zoom)
    lon = x / n * 360.0 - 180.0
    lat = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    lat = lat * 180.0 / math.pi
    return lon, lat


def cal_tiff_box(x1, y1, x2, y2, zoom):
    LT = xyz2lonlat(x1, y1, zoom)
    RB = xyz2lonlat(x2 + 1, y2 + 1, zoom)
    return LT[0], RB[1], RB[0], LT[1]

In [37]:
# configs
Path = '/Users/cubics/Geospatial_Data_Downloader/Google_Map_Tiles/data/'
point_lt = (110.24582, 37.62737)
point_rb = (110.37745, 37.53976)
zoom = 16
point_lt_x, point_lt_y = lonlat2xyz(point_lt[0], point_lt[1], zoom)
point_rb_x, point_rb_y = lonlat2xyz(point_rb[0], point_rb[1], zoom)

In [38]:
# batch download with tqdm
with tqdm.tqdm(total=(point_rb_x - point_lt_x + 1) * (point_rb_y - point_lt_y + 1)) as pbar:
    for x in range(point_lt_x, point_rb_x + 1):
        for y in range(point_lt_y, point_rb_y + 1):
            url = build_url(x, y, zoom)
            path = Path + str(zoom) + '/' + str(x) + '/'
            if not os.path.exists(path):
                os.makedirs(path)
            if os.path.exists(path + str(y) + '.png'):
                pbar.update(1)
                continue
            else:     
                r = requests.get(url,verify=False)
                with open(path + str(y) + '.png', 'wb') as f:
                    f.write(r.content)
                pbar.update(1)

100%|██████████| 550/550 [00:00<00:00, 4396.38it/s]


In [39]:
# merge tiles
tiles = []
for x in range(point_lt_x, point_rb_x + 1):
    row = []
    for y in range(point_lt_y, point_rb_y + 1):
        img = cv2.imread(Path + str(zoom) + '/' + str(x) + '/' + str(y) + '.png')
        row.append(img)
    tiles.append(row)
tiles = np.array(tiles)
img = np.hstack([np.vstack(row) for row in tiles])
cv2.imwrite(Path + str(zoom) + '/merged.png', img)

True

In [41]:
# using gdal to convert to tiff
os.system('gdal_translate -of GTiff -a_srs EPSG:4326 -a_ullr {x1} {y1} {x2} {y2} {src} {dst}'.format(
    x1=cal_tiff_box(point_lt_x, point_lt_y, point_rb_x, point_rb_y, zoom)[0],
    y1=cal_tiff_box(point_lt_x, point_lt_y, point_rb_x, point_rb_y, zoom)[1],
    x2=cal_tiff_box(point_lt_x, point_lt_y, point_rb_x, point_rb_y, zoom)[2],
    y2=cal_tiff_box(point_lt_x, point_lt_y, point_rb_x, point_rb_y, zoom)[3],
    src=Path + str(zoom) + '/merged.png',
    dst=Path + str(zoom) + '/merged.tif'
))

Input file size is 6400, 5632
0...10...20...30...40...50...60...70...80...90...100 - done.


0